In [3]:
import requests

url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

response = requests.get(url)
if response.status_code == 200:
    print("下載成功")

下載成功


In [4]:
response.encoding

In [7]:

print(type(response.json()))
response.json()

<class 'dict'>


{'cwbopendata': {'@xmlns': 'urn:cwb:gov:tw:cwbcommon:0.1',
  'identifier': 'a9513e4d-f1eb-59dd-33cd-f9e42ea5fd8b',
  'sender': 'weather@cwb.gov.tw',
  'sent': '2023-08-12T03:30:03+08:00',
  'status': 'Actual',
  'msgType': 'Issue',
  'source': 'MFC',
  'dataid': 'C0032-001',
  'scope': 'Public',
  'dataset': {'datasetInfo': {'datasetDescription': '三十六小時天氣預報',
    'issueTime': '2023-08-12T05:00:00+08:00',
    'update': '2023-08-12T03:30:03+08:00'},
   'location': [{'locationName': '臺北市',
     'weatherElement': [{'elementName': 'Wx',
       'time': [{'startTime': '2023-08-12T06:00:00+08:00',
         'endTime': '2023-08-12T18:00:00+08:00',
         'parameter': {'parameterName': '多雲午後短暫雷陣雨', 'parameterValue': '22'}},
        {'startTime': '2023-08-12T18:00:00+08:00',
         'endTime': '2023-08-13T06:00:00+08:00',
         'parameter': {'parameterName': '多雲短暫陣雨', 'parameterValue': '8'}},
        {'startTime': '2023-08-13T06:00:00+08:00',
         'endTime': '2023-08-13T18:00:00+08:00',


In [1]:
import requests
def download_data()->dict:
    url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

    response = requests.get(url)
    if response.status_code == 200:
        print("下載成功")
    return response.json()

In [2]:
def jsonDict_csvList(json)->list[dict]:
    '''
    - 傳入josn的資料結構
    - 取出需要的資料
    - 組合成list[dict]
    '''
    location = json['cwbopendata']['dataset']['location']
    weather_list = []
    for item in location:
        city_item = {}
        city_item['城市'] = item['locationName']
        city_item['啟始時間'] = item['weatherElement'][1]['time'][0]['startTime']
        city_item['結束時間'] = item['weatherElement'][1]['time'][0]['endTime']
        city_item['最高溫度'] = float(item['weatherElement'][1]['time'][0]['parameter']['parameterName'])
        city_item['最低溫度'] = float(item['weatherElement'][2]['time'][0]['parameter']['parameterName'])
        city_item['感覺'] = item['weatherElement'][3]['time'][0]['parameter']['parameterName']
        weather_list.append(city_item)
    return weather_list

In [4]:
import csv
def save_csv(data:list[dict],fileName) -> None:
    '''
    - 將list[dict]儲存
    - 參數fileName要儲存的檔案名
    '''
    with open(fileName,mode='w',encoding='utf-8',newline='') as file:
        fieldnames = ['城市', '啟始時間','結束時間','最高溫度','最低溫度','感覺']
        writer = csv.DictWriter(file,fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

In [5]:
json_data = download_data() #下載資料
csv_list = jsonDict_csvList(json_data) #解析json的資料成為csv的結構
save_csv(csv_list,"今天天氣.csv") #儲存資料 

下載成功


In [13]:
from datetime import datetime
import pytz #要換成自己的時區，不然會抓server所在地點的時間
tw_timezone = pytz.timezone('Asia/Taipei')
current_time = datetime.now(tw_timezone)
 
print(f"{current_time.year}年{current_time.month}月{current_time.day}日")

2023年8月12日
